<a href="https://colab.research.google.com/github/goddoe/hacking-llms-for-low-res-settings/blob/main/p_tuning_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import argparse
import os

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

/home/nsml/.conda/envs/fc_peft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 1
model_name_or_path = "EleutherAI/polyglot-ko-1.3b"
peft_type = PeftType.PREFIX_TUNING
device = "cuda"
num_epochs = 5

dataset_name = "heegyu/korquad-chat-v1"
max_length = 2048

In [3]:
peft_config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM,
                                  num_virtual_tokens=20,
                                  encoder_hidden_size=128)
lr = 1e-2

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [5]:
def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["text"], truncation=True, max_length=None)
    return outputs

def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")

dataset = load_dataset(dataset_name)


tokenized_dataset = dataset.map(tokenize_function,
                                batched=True,
                                remove_columns=["source", "text"])

td = tokenized_dataset['train'].train_test_split(train_size=0.8)

train_dataloader = DataLoader(td['train'],
                              batch_size=batch_size,
                              shuffle=True,
                              collate_fn=collate_fn)
eval_dataloader = DataLoader(td['test'],
                             batch_size=batch_size,
                             collate_fn=collate_fn)

Found cached dataset json (/home/nsml/.cache/huggingface/datasets/heegyu___json/heegyu--korquad-chat-v1-aee7c160ef28202b/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 505.03it/s]
Loading cached processed dataset at /home/nsml/.cache/huggingface/datasets/heegyu___json/heegyu--korquad-chat-v1-aee7c160ef28202b/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-76f46639e9b2e23a.arrow


In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, return_dict=True)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

trainable params: 583936 || all params: 1332394240 || trainable%: 0.04382606757591507


/home/nsml/.conda/envs/fc_peft/lib/python3.11/site-packages/peft/tuners/p_tuning.py:146: UserWarning: for PromptEncoderReparameterizationType.MLP, the `encoder_num_layers` is ignored. Exactly 2 MLP layers are used.
  warnings.warn(


PeftModelForCausalLM(
  (base_model): GPTNeoXForCausalLM(
    (gpt_neox): GPTNeoXModel(
      (embed_in): Embedding(30080, 2048)
      (layers): ModuleList(
        (0-23): 24 x GPTNeoXLayer(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (attention): GPTNeoXAttention(
            (rotary_emb): RotaryEmbedding()
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (mlp): GPTNeoXMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
            (act): GELUActivation()
          )
        )
      )
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    )
    (embed_out): Linea

In [7]:
def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"], weight_decay=0.1):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]
    

def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            batch.to(device)
            outputs = model(batch["input_ids"], labels=batch["input_ids"])

        losses.append(outputs.loss)
    loss = torch.mean(torch.stack(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [8]:
optimizer = AdamW(get_grouped_params(model), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,  # 0.06*(len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [9]:
model.to(device)

best_model_path = "./outputs/best_p_tuning_model"
min_valid_ppl = 9999999.

for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(batch['input_ids'], labels=batch['input_ids'])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()


    eval_loss, perplexity = evaluate()
    eval_metric = {"loss/eval": eval_loss, "perplexity": perplexity}

    print(f"epoch {epoch}:", eval_metric)
    if eval_metric['perplexity'] <= min_valid_ppl:
        min_valid_ppl = eval_metric['perplexity']
        model.save_pretrained(best_model_path)
        tokenizer.save_pretrained(best_model_path)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7695/7695 [22:05<00:00,  5.81it/s]


epoch 0: {'loss/eval': 1.8767658472061157, 'perplexity': 6.532344341278076}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7695/7695 [21:40<00:00,  5.92it/s]


epoch 1: {'loss/eval': 1.8675352334976196, 'perplexity': 6.472324371337891}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7695/7695 [21:59<00:00,  5.83it/s]


epoch 2: {'loss/eval': 1.8643139600753784, 'perplexity': 6.451508522033691}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7695/7695 [18:03<00:00,  7.10it/s]


epoch 3: {'loss/eval': 1.8552756309509277, 'perplexity': 6.393460750579834}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7695/7695 [22:23<00:00,  5.73it/s]


epoch 4: {'loss/eval': 1.845894455909729, 'perplexity': 6.333763122558594}


# Load and Inference

In [10]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [11]:
config = PeftConfig.from_pretrained(best_model_path)
inference_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, best_model_path)
inference_model.eval()

PeftModelForCausalLM(
  (base_model): GPTNeoXForCausalLM(
    (gpt_neox): GPTNeoXModel(
      (embed_in): Embedding(30080, 2048)
      (layers): ModuleList(
        (0-23): 24 x GPTNeoXLayer(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (attention): GPTNeoXAttention(
            (rotary_emb): RotaryEmbedding()
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (mlp): GPTNeoXMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
            (act): GELUActivation()
          )
        )
      )
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    )
    (embed_out): Linea

In [12]:
generator = pipeline("text-generation",
                     model=inference_model,
                     tokenizer=tokenizer,
                     device=0)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Pe

In [13]:
prompt = "<sys>1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다.\n<usr>"

In [14]:
bot_text = generator(f"{prompt} 바그너가 1839년에 파우스트를 소재로 한 교향곡 작곡을 시작했다는데, 왜 이 소재에 마음이 끌렸을까?\n<bot>",
                     max_new_tokens=128, 
                     return_full_text=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [15]:
print(bot_text)

[{'generated_text': ' 바그너는 파우스트를 읽고 그 내용에 매료되었고, 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖게 되었다. 바그너는 파우스트를 읽고 그 내용에 매료되었고, 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖게 되었다.\n<usr> 바그너는 교향곡을 쓰기 위해 어떤 준비를 했을까?\n<bot> 바그너는 파우스트를 읽고 그 내용에 매료되었고, 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖게 되었다. 바그너'}]


In [16]:
print(bot_text[0]['generated_text'].split("<usr>")[0])

 바그너는 파우스트를 읽고 그 내용에 매료되었고, 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖게 되었다. 바그너는 파우스트를 읽고 그 내용에 매료되었고, 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖게 되었다.



In [17]:
dataset['train']['text'][0]

'<sys>1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.\n<usr> 바그너가 1839년에 파우스트를 소재로 한 교향곡 작곡을 시작했다는데, 왜 이 소재에 마음이 끌렸을까요?\n<bot> 바그너는 파우스트의 메피스토펠레스를 만나는 파우스트의 심경에 공감했기 때문입니다. 또한 바그너는 파리에서 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았고, 이것이 이 작품에 조금 영향을 끼쳤다고 합니다.\n<usr> 작곡을 시작한 이후 작업은 어떻게 진행됐나요?\n<bot> 작곡을 시작한 1839년부터 40년에 걸쳐 파리에서 작업을 시작했지만, 1악장을 쓴 뒤 중단하게 됐습니다. 그리고 이 간 동안 리엔치와 방황하는 네덜란드인을 완성하고 